# Vaex Run

In [1]:
import os
import vaex as vx
import time
import numpy as np
import shutil
import gc
import joblib
import psutil

In [2]:
# PATH_DATA = '../../df-showdown-data/'
PATH_DATA = 'O:/df-showdown-data/'
TEMP_PATH = PATH_DATA+'temp/'
PATH_OUTPUT = '../output2/'

SLA_NAME = 'vaex-sla.pkl'
MEM_NAME = 'vaex-mem.pkl'


In [3]:
all_sla = {}
all_memory = {}

In [4]:
def get_memory():
    est_memory = psutil.virtual_memory()[3]/1000000000
    return est_memory

##     Task1 & Task2- I/O

In [5]:
all_sla['task1'] = {}
all_sla['task1']['csv'] = {}
all_sla['task1']['prq'] = {}

all_sla['task2'] = {}
all_sla['task2']['csv'] = {}
all_sla['task2']['prq'] = {}



all_memory['task1'] = {}
all_memory['task1']['csv'] = {}
all_memory['task1']['prq'] = {}

all_memory['task2'] = {}
all_memory['task2']['csv'] = {}
all_memory['task2']['prq'] = {}


In [6]:
for i in range(10):
    print(i)
    if i<=5:
        fn = 'data_{}.csv'.format(i)
        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1 = vx.open(PATH_DATA+fn)
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        nrows = str(int(dtemp1.shape[0]/1000000))
        all_sla['task1']['csv'][nrows]=end
        all_memory['task1']['csv'][nrows]=end_mem

        start = time.perf_counter()
        start_mem = get_memory()
        dtemp1.export(TEMP_PATH + 'temp.csv')
        end_mem = get_memory() - start_mem
        end = time.perf_counter() - start
        all_sla['task2']['csv'][nrows]=end
        all_memory['task2']['csv'][nrows]=end_mem

        del dtemp1
        time.sleep(3)
        gc.collect()
        os.remove(TEMP_PATH + 'temp.csv')
        time.sleep(2)

    fn = 'data_{}.parquet'.format(i)
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1 = vx.open(PATH_DATA+fn)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    all_sla['task1']['prq'][nrows]=end
    all_memory['task1']['prq'][nrows]=end_mem
    
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1.export(TEMP_PATH + 'temp.parquet')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    all_sla['task2']['prq'][nrows]=end
    all_memory['task2']['prq'][nrows]=end_mem

    del dtemp1
    time.sleep(3)
    gc.collect()
    os.remove(TEMP_PATH + 'temp.parquet')

0
1


In [7]:
all_sla

{'task1': {'csv': {'0': 1.7250648999999996, '1': 0.22318999999999534},
  'prq': {'0': 0.15654809999999486, '1': 0.02502590000000282}},
 'task2': {'csv': {'0': 0.7188467000000003, '1': 4.6817655999999985},
  'prq': {'0': 0.13102400000000358, '1': 0.6557847999999993}}}

In [8]:
all_memory

{'task1': {'csv': {'0': 0.09196748800000165, '1': 0.05468979199999957},
  'prq': {'0': -0.0006840319999987798, '1': -0.004706303999999051}},
 'task2': {'csv': {'0': 0.04147609600000024, '1': 0.11171839999999911},
  'prq': {'0': 0.0007045120000004346, '1': 0.06041190399999863}}}

In [14]:
# del dtemp1
# gc.collect()

## Get Data for other task

In [9]:
all_df = [vx.open(PATH_DATA+'data_{}.parquet'.format(i)) for i in [4,5,6,7]]
# all_df = [vx.open(PATH_DATA+'data_{}.parquet'.format(i)) for i in [1,2]]
df_right = vx.open(PATH_DATA+'data_to_join.parquet')

In [10]:
len(all_df)

2

## Task3 - Sorting

In [11]:
n_task = 'task3'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [12]:
# dtmp = all_df[0]
# dtmp.sort

In [13]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.sort(['Date','Amount'],ascending=[True,False])
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()


1.03
5.03


In [14]:
all_sla[n_task]

{'1': 0.29613539999999716, '5': 1.8192484000000064}

In [15]:
all_memory[n_task]

{'1': -0.0002621440000005748, '5': 0.06839910399999916}

## Task4 - Filtering

In [16]:
n_task = 'task4'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [17]:
# dtemp1 = all_df[0]
filt1 = ['Shipped - Delivered to Buyer', 'Cancelled']
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1[(dtemp1['Amount']>300) & (~dtemp1['Status'].isin(filt1))]
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()
    


1.03
5.03


In [18]:
all_sla[n_task]

{'1': 0.21465299999999843, '5': 0.17182959999999525}

In [19]:
all_memory[n_task]

{'1': 0.01178828799999998, '5': 0.04856217600000079}

## Task5 - Merging

In [20]:
n_task = 'task5'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [21]:
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    dtemp1['key'] = dtemp1['Date'] +'_'+dtemp1['ship-service-level']
    df_right['key'] = df_right['Date'] +'_'+df_right['ship-service-level']
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.join(df_right,on='key',how='left',rsuffix='_R')
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))

    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03


In [22]:
all_sla[n_task]

{'1': 0.2704497999999944, '5': 0.7664309000000173}

In [23]:
all_memory[n_task]

{'1': 0.0016957440000009427, '5': 0.14607155200000044}

## Task6 - udf apply

In [24]:
n_task = 'task6'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [25]:
def fun(x):
    x0 = x + 1
    for i in range(25):
        if x0<800:
            x0 += i
            x0 = (x0/3.0)*2.5
            x0 = x0*1.2
        else:
            x0 += i/2.0
            x0 = (x0/4.0)*3.8

    return x0

In [26]:
# dtemp1=all_df[0]
# dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp1['Amount2'] = dtemp1['Amount'].apply(fun)
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem
    
    del dtemp1
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03


In [27]:
all_sla[n_task]

{'1': 0.010548099999994065, '5': 0.01299320000001103}

In [28]:
all_memory[n_task]

{'1': -0.00018841600000030212, '5': 0.0}

In [34]:
# type(all_df[0])

## Task7 - aggregation

In [35]:
n_task = 'task7'
all_sla[n_task] = {}
all_memory[n_task] = {}

In [36]:
def p25(x):
    return np.percentile(x,25)

def p75(x):
    return np.percentile(x,75)

In [37]:
# dtemp1=all_df[0]
for dtemp1 in all_df:
    print(round(len(dtemp1)/1000000,2))
    start = time.perf_counter()
    start_mem = get_memory()
    dtemp2 = dtemp1.groupby(['Date','Status']).agg({'Amount':'mean'})
    end_mem = get_memory() - start_mem
    end = time.perf_counter() - start
    nrows = str(int(dtemp1.shape[0]/1000000))
    
    all_sla[n_task][nrows]=end
    all_memory[n_task][nrows]=end_mem

    del dtemp1
    del dtemp2
    time.sleep(4)
    gc.collect()
    time.sleep(4)
    gc.collect()

1.03
5.03


In [38]:
all_sla[n_task]

{'1': 0.9097418000000062, '5': 2.526093000000003}

In [39]:
all_memory[n_task]

{'1': 0.0028999680000012518, '5': -0.08529920000000146}

## Save results

In [40]:
joblib.dump(all_sla,PATH_OUTPUT+SLA_NAME)
joblib.dump(all_memory,PATH_OUTPUT+MEM_NAME)

['../output/vaex-mem.pkl']